In [63]:
import os,sys
sys.path.append(os.path.abspath('./../../'))
from Controllers.MarkedCellController import MarkedCellController,CellSources
from Controllers.SqlController import SqlController
from model.annotation_points import MarkedCellView
from sqlalchemy import inspect
inst = inspect(MarkedCellView)
attr_names = [c_attr.key for c_attr in inst.mapper.column_attrs]
import pandas as pd
from cell_extractor.DetectorMetrics import AnnotationProximityTool
from cell_extractor.CellDetector import CellDetector
import numpy as np
from collections import Counter
user_look_up = {38:'Marissa',41:'Julian',2:'Beth'}

def get_DataFrame_from_query_result(results,category,factor):
    values = []
    for i in results:
        source = i.source.value
        if '-' in source:
            source = source.split('-')[1]    
        x,y,z = np.array([i.x,i.y,i.z]).astype(float)/factor
        values.append([x,y,z,f'{category}_{user_look_up[i.FK_annotator_id]}_{source}'])
    # values = [[eval(f'j.{i}')  for j in results ] for i in attr_names]
    df = pd.DataFrame(dict(zip(['x','y','section','name'],np.array(values).T)))
    df["x"] = pd.to_numeric(df["x"])
    df["y"] = pd.to_numeric(df["y"])
    df["section"] = pd.to_numeric(df["section"])
    return df

def get_DataFrame_from_detection_df(df,label):
    data = np.array([df.col.to_numpy().astype(int),df.row.to_numpy().astype(int),\
        df.section.to_numpy().astype(int)])
    data = pd.DataFrame(data.T,columns=['x','y','section'])
    data['name'] = [label]*len(df)
    return data

In [34]:
animal = 'DK55'
controller = SqlController()
factor = controller.get_resolution(animal)
controller = MarkedCellController()

In [35]:
search_dict = {'FK_prep_id':animal,'FK_cell_type_id':5}
qcs = controller.get_marked_cells(search_dictionary=search_dict)
positives = [i for i in qcs if i.source == CellSources.HUMAN_POSITIVE]
negatives = [i for i in qcs if i.source == CellSources.HUMAN_NEGATIVE]
positives = get_DataFrame_from_query_result(positives,'Round1',factor)
negatives = get_DataFrame_from_query_result(negatives,'Round1',factor)
qcs = get_DataFrame_from_query_result(qcs,'Round1',factor) 

In [64]:
detector = CellDetector(animal)
detections = detector.load_detections()
sures = detections[detections.predictions==2]
unsures = detections[detections.predictions==0]
sures = get_DataFrame_from_detection_df(sures,'sures')
unsures = get_DataFrame_from_detection_df(unsures,'unsures')

In [65]:
tool = AnnotationProximityTool()
tool.pair_distance=0.1
tool.set_annotations_to_compare(pd.concat([sures,unsures,qcs]))
tool.find_equivalent_points()
Counter([tuple(i) for i in tool.pair_categories.values()]).most_common()

calculating distance matrix
